In [9]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertMainLayer
from gensim.models import KeyedVectors

In [2]:
model = tf.keras.models.load_model('bert_model.h5', custom_objects={
                                   'TFBertMainLayer': TFBertMainLayer})


2021-11-18 23:25:09.213504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 23:25:09.221108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 23:25:09.221303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 23:25:09.221929: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 22)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 22)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 22,                                            

In [15]:
w2v = KeyedVectors.load_word2vec_format(
    'data/GloVe/glove.6B.50d_gensim.txt', binary=False, unicode_errors='ignore')

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
seq_length = 20


def tokenize_input(text):
    tokens = tokenizer.encode_plus(text, max_length=seq_length+2, truncation=True,
                                   padding='max_length', add_special_tokens=True, return_tensors='tf')
    return {
        'input_ids': tf.cast(tokens['input_ids'], tf.float64),
        'attention_mask': tf.cast(tokens['attention_mask'], tf.float64),
    }


In [21]:
text = tokenize_input('This is my finest work! I would like to thank my')

In [22]:
with tf.device('/cpu:0'):
    props = model.predict(text)

In [23]:
props[0]

array([ 2.4269433e-01,  1.1603429e-01, -8.0039136e-02, -1.6732435e-01,
        2.3914970e-01,  1.9934663e-01, -4.5312858e-01,  2.2180339e-02,
       -4.7225446e-02,  5.9892476e-02,  1.5947868e-01,  1.8863934e-01,
       -3.8892931e-01, -1.7443441e-01,  2.5528559e-01,  3.5048008e-02,
        1.2196335e-01,  6.0894001e-02, -3.8234696e-01, -2.1086949e-01,
       -3.0039227e-02,  2.1462773e-01,  4.2740848e-02, -4.1153636e-02,
        2.4138035e-01, -1.4767510e+00, -3.6957899e-01,  5.2965846e-02,
        2.0505849e-01, -1.6708064e-01,  2.9095948e+00,  1.2955153e-02,
       -2.1322921e-01, -3.4871867e-03,  5.8057465e-02,  2.8282886e-02,
        7.2933048e-02,  1.2551227e-01,  7.2577752e-02,  2.4644399e-02,
        4.9227723e-03,  1.2666218e-01, -1.9744448e-03, -2.1541884e-02,
       -9.4300590e-02,  2.9723583e-02, -1.9477072e-01, -1.8032485e-01,
        5.1644225e-02, -5.5311400e-02], dtype=float32)

In [24]:
w2v.similar_by_vector(props[0], topn=10)

[('.', 0.9491175413131714),
 ('one', 0.9488591551780701),
 ('as', 0.9465561509132385),
 ('but', 0.9450469613075256),
 ('well', 0.9383603930473328),
 ('same', 0.9374268651008606),
 ('it', 0.937322199344635),
 ('only', 0.9251962900161743),
 ('this', 0.9248157739639282),
 ('another', 0.9156841039657593)]